In [5]:
from pyspark import SparkContext, SparkConf
import numpy as np

In [14]:
conf = SparkConf()
sc = SparkContext().getOrCreate(conf = conf)

In [15]:
def page_rank(data_path, iterations, beta):
    dataset = sc.textFile(data_path).map(lambda v: v.split("\t"))
    Deg = dataset.groupByKey().mapValues(list) #This gets all the outgoing nodes into a group and sorts
    Deg = Deg.map(lambda w: (int(w[0]),set(int(i) for i in w[1]))).sortByKey() #Removing the duplicates values and converting into integer
    values = Deg.count() #getting the count of nodes 
#This is my RDD M martix where each row is a column in theortical M matrix
    M = Deg.map(lambda w : np.array([(1/len(w[1]) if (i+1 in w[1]) else 0)for i in range(values)]))
    
#intializing the r(old) with 1/n
    r = np.zeros(values)
    r.fill(1/values)
    
# one is my 1 of size(n X 1) and transport is this whole term (1-beta)/n*1
    one = np.ones(values)
    transport = ((1 - beta)/values)*one
    q = M.zipWithIndex() #adds index to every row in RDD
    
    for i in range(iterations):
        print("Iteration: ", i+1)
        q = M.zipWithIndex()
        q = q.map(lambda w: (w +(r[w[1]],))) #adding rank value to each row in RDD
        w = q.map(lambda w: (1, w[0]*w[2])) #multipying the r1 with M column
        e = w.reduceByKey(lambda n1, n2 : n1 + n2) #adding all the rows of my M gives me this term (beta*M*r)
        print(e.take(1))
        e = e.map(lambda w: transport + w[1]*beta) #multiplying the probability and adding the transport vector
        print(e.take(1))
        r = np.array(e.collect()[0]) #collecting my (r new)
        p = np.argsort(r) # sorting them by argument to print the top ranks
        top5 = list(p[-5:])
        for i in range(5):
            g = top5.pop()
            print("{}:{}".format(g+1,r[g]))
    return r

In [16]:
r = page_rank("graph-small.txt", 1, 0.8)
p = np.argsort(r)
bottom5 = list(p[:5])
top5 = list(p[-5:])

print("Top 5 values:")
for i in range(5):
    g = top5.pop()
    print("{}:{}".format(g+1,r[g]))

print("Bottom 5 Values:")
for i in range(5):
    g = bottom5.pop()
    print("{}:{}".format(g+1,r[g]))
    
sc.stop()

Iteration:  1
[(1, array([0.03192576, 0.00343452, 0.00759199, 0.00642039, 0.0163079 ,
       0.00518687, 0.00365909, 0.00522619, 0.01533507, 0.00485354,
       0.00340115, 0.00603175, 0.02065115, 0.04198135, 0.00368687,
       0.00737034, 0.00729167, 0.01605664, 0.00378943, 0.00444755,
       0.0055636 , 0.01653125, 0.00274242, 0.00753968, 0.00698413,
       0.01636158, 0.03495931, 0.00385844, 0.00796429, 0.0077061 ,
       0.01618059, 0.00444444, 0.00503968, 0.00635256, 0.01589134,
       0.00416306, 0.00363095, 0.00792832, 0.01723821, 0.04090808,
       0.00435354, 0.00739358, 0.00632479, 0.01667067, 0.00352474,
       0.0040609 , 0.0061271 , 0.01997197, 0.00418651, 0.00424242,
       0.00696465, 0.01657789, 0.04151848, 0.00642857, 0.0095202 ,
       0.00690152, 0.01710329, 0.00396368, 0.00395833, 0.0065803 ,
       0.01577846, 0.00440909, 0.00364358, 0.00920375, 0.01711681,
       0.03837941, 0.00296429, 0.00574242, 0.00687771, 0.01359246,
       0.0045202 , 0.00344755, 0.00738387, 